In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV,train_test_split
import dagshub
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [3]:
dagshub.init(repo_owner='akshatsharma2407', repo_name='GMC_motors', mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/akshatsharma2407/GMC_motors.mlflow')

Accessing as akshatsharma2407

Initialized MLflow to track repo "akshatsharma2407/GMC_motors"

Repository akshatsharma2407/GMC_motors initialized!

In [4]:
mlflow.autolog()
mlflow.set_experiment(experiment_name='RF_BASE')
mlflow.start_run()

2025/03/11 11:25:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/11 11:25:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/03/11 11:25:40 INFO mlflow.tracking.fluent: Experiment with name 'RF_BASE' does not exist. Creating a new experiment.


<ActiveRun: >

In [5]:
df = pd.read_csv('C:/Users/aksha/Downloads/CLEANED_GMC_DIESEL.csv')

df.drop(columns=['PRICE RANGE','MAKE ORIGIN','PARENT COMPANY','IMAGE','BRAND'],inplace=True)

df['AGE OF CAR'] = df['AGE OF CAR'].astype(str)
df['MODEL'] = df['MODEL'].astype(str)

df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

xtrain,xtest,ytrain,ytest = train_test_split(df.drop(columns='PRICE($)'),df['PRICE($)'],test_size=0.2)

In [24]:
ct1 = ColumnTransformer(
            [
                ('RatingImputer',SimpleImputer(missing_values=-1,strategy='mean'),['RATING']),
                ('OHE',ce.TargetEncoder(verbose=1),['CAR NAME','MODEL/CLASS','DEALER NAME','DEALER LOCATION (CITY)','DEALER LOCATION (STATE)']),
                ('OE',OrdinalEncoder(categories=
                                    [
                                        ["1937", "1951", "1952", "1966", "1968", "1977", "1979", "1984", "1986", "1987", 
                "1988", "1989", "1996", "1998", "1999", "2000", "2001", "2002", "2003", "2004", 
                "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", 
                "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"],
            ['Used','GMC Certified','New'],
            ["87", "73", "72", "58", "56", "47", "45", "40", "38", "37", "36", "35", "28", 
                "26", "25", "24", "23", "22", "21", "20", "19", "18", "17", "16", "15", "14", 
                "13", "12", "11", "10", "9", "8", "7", "6", "5", "4", "3", "2", "1", "0"]
            ]
            ),['MODEL','STOCK TYPE','AGE OF CAR'])
            ],
            remainder='passthrough'
        )


ct2 = ColumnTransformer(
            [
                ('stdscaler',StandardScaler(),slice(0,13))
            ]
        )

In [30]:
pipe = Pipeline([
    ('ct1',ct1),
    ('ct2',ct2),
    ('model',RandomForestRegressor())
])

In [31]:
pipe.set_output(transform='pandas')

Pipeline(steps=[('ct1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('RatingImputer',
                                                  SimpleImputer(missing_values=-1),
                                                  ['RATING']),
                                                 ('OHE',
                                                  TargetEncoder(verbose=1),
                                                  ['CAR NAME', 'MODEL/CLASS',
                                                   'DEALER NAME',
                                                   'DEALER LOCATION (CITY)',
                                                   'DEALER LOCATION (STATE)']),
                                                 ('OE',
                                                  OrdinalEncoder(categories=[['1937',
                                                                              '1951',
                                                                              '1952',
                                                                              '1966',
                                                                              '1968',
                                                                              '1977',
                                                                              '1979',
                                                                              '1984',...
                                                                             ['Used',
                                                                              'GMC '
                                                                              'Certified',
                                                                              'New'],
                                                                             ['87',
                                                                              '73',
                                                                              '72',
                                                                              '58',
                                                                              '56',
                                                                              '47',
                                                                              '45',
                                                                              '40',
                                                                              '38',
                                                                              '37',
                                                                              '36',
                                                                              '35',
                                                                              '28',
                                                                              '26',
                                                                              '25',
                                                                              '24',
                                                                              '23',
                                                                              '22',
                                                                              '21',
                                                                              '20',
                                                                              '19',
                                                                              '18',
                                                                              '17',
                                                                              '16',
                                                                              '15',
                                                                         

In [ ]:
pipe.fit(xtrain,ytrain)

ypred = pipe.predict(xtest)

print('mean squared error',mean_squared_error(ytest,ypred))
print('mean_absolute_error',mean_absolute_error(ytest,ypred))
print('r2_score',r2_score(ytest,ypred))

2025/03/11 11:37:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/03/11 11:37:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\aksha\AppData\Lo

In [ ]:
mlflow.end_run()

🏃 View run incongruous-ox-941 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/6/runs/67b05a32212b42f0af2bae36b828e09f
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/6
